In [1]:
"""This is a tool box for the Two Box Climate Scheme modlue. In this tool box are functions used for calculating
equilibrium position and other quantities associated with the two box model created in the 2018-2019 work of Sura & 
Sweeney"""


#BEGIN CODE


def linear_equilibrium(mu1, mu2, complex_guess_1, complex_guess_2):
    """The linear equilibrium function finds a two dimensional vector solution. The solutions we are looking for are real.
    Many solutions that can be found are complex and because of the numerical methods in which python finds these solutions
    we have decided to use the mpmath modules. This method of finding the solution for our non linear heat flux uses
    the muller method of numerical calculation. For more information on mpmath root finding: 
    http://mpmath.org/doc/1.1.0/calculus/optimization.html"""
    import numpy as np
    import sympy
    import mpmath
    #List of Constants
    I0 = 1367.0 #Solar constant W/m^2
    a = 2.8 #Unitless parameter needed for linear albedo feedback relationships more info in Budyko 1969
    b=.009 #Another parameter needed for linear feedback relationship more info in Budyko 1969
    sig = 5.67*10**-8 #Stephan boltzmann constant m^2 kg s^-2 K^-1
    e = .64 #Emmisivity of earth
    A = 600 #Heat flux parameter for non linear heat forcing
    solution_array = []
    for x in mu1:
        for y in mu2:
            #Below we find the two-dimensional vector solutions in form of [polar_solution, tropical_solution]
            f = [lambda T1,T2: (.156*x*I0*(1-a) + .156*x*I0*b*T1 - e*sig*(T1**4)+(A*(T2 - T1))), 
                 lambda T1,T2: (.288*y*I0*(1-a) + .288*y*I0*b*T2 - e*sig*(T2**4)+(A*(T1 - T2)))]
            solution = np.array(mpmath.findroot(f, ([complex_guess_1, complex_guess_2]), 
                                                solver='muller', verify = False))
            if abs(mpmath.im(solution[0])) > 1e-10:
                solution[0] = np.nan
            if abs(mpmath.im(solution[1])) > 1e-10:
                solution[1] = np.nan
            solution_array.append([mpmath.re(solution[0]), mpmath.re(solution[1]), x, y])
    return(solution_array)



def nonlinear_equilibrium(mu1, mu2, complex_guess_1, complex_guess_2):
    """The nonlinear equilibrium function finds a two dimensional vector solution. The solutions we are looking for are real.
    Many solutions that can be found are complex and because of the numerical methods in which python finds these solutions
    we have decided to use the mpmath modules. This method of finding the solution for our non linear heat flux uses
    the muller method of numerical calculation. For more information on mpmath root finding: 
    http://mpmath.org/doc/1.1.0/calculus/optimization.html"""
    import numpy as np
    import sympy
    import mpmath
    #List of Constants
    I0 = 1367.0 #Solar constant W/m^2
    a = 2.8 #Unitless parameter needed for linear albedo feedback relationships more info in Budyko 1969
    b=.009 #Another parameter needed for linear feedback relationship more info in Budyko 1969
    sig = 5.67*10**-8 #Stephan boltzmann constant m^2 kg s^-2 K^-1
    e = .64 #Emmisivity of earth
    A = 600 #Heat flux parameter for non linear heat forcing
    solution_array = []
    for x in mu1:
        for y in mu2:
            #Below we find the two-dimensional vector solutions in form of [polar_solution, tropical_solution]
            f = [lambda T1,T2: (.156*x*I0*(1-a) + .156*x*I0*b*T1 - e*sig*(T1**4)+(A/(T2 -T1))), 
                 lambda T1,T2: (.288*y*I0*(1-a) + .288*y*I0*b*T2 - e*sig*(T2**4)+(A/(T1 -T2)))]
            solution = np.array(mpmath.findroot(f, ([complex_guess_1, complex_guess_2]), 
                                                solver='muller', verify = False))
            if abs(mpmath.im(solution[0])) > 1e-10:
                solution[0] = np.nan
            if abs(mpmath.im(solution[1])) > 1e-10:
                solution[1] = np.nan
            solution_array.append([mpmath.re(solution[0]), mpmath.re(solution[1]), x, y])
    return(solution_array)

    
def linear_albedo_feedback(k_vector, mu1, mu2):
    """To create the linear albedo feedback we follow the methods used in the 1978 fraedrich paper, assinging 
    an 'a' parameter value of .75 and finding solutions to the system of differential equations.
    The linear albedo feedback produced only real solutions. Because of this, we may take advantage of the speeds
    of computation provided through the scipy package because we do not need to search for nonreal answers to 
    eliminate. For more information: https://docs.scipy.org/doc/scipy/reference/optimize.html"""
    import numpy as np
    from scipy.optimize import fsolve
    #List of Constants
    I0 = 1367.0 #Solar constant W/m^2
    a = .75 #Unitless parameter needed for linear albedo feedback relationships more info in Budyko 1969
    sig = 5.67*10**-8 #Stephan boltzmann constant m^2 kg s^-2 K^-1
    e = .64 #Emmisivity of earth
    T1 = k_vector[0]
    T2 = k_vector[1]
    temp = np.zeros(2)
    temp[0] = (.156*mu1*I0*(1-.75) - e*sig*(T1**4))*(1.0*10**-8)
    temp[1] = (.288*mu2*I0*(1-.75) - e*sig*(T2**4))*(1.5*10**-8)
    """    deepfreeze = []
    for x in mu1:
        for y in mu2:
            solution = fsolve(linear_abledo_feedback, k_vector, args =(x,y))
            deepfreeze.append([solution[0], solution[1], x, y])"""
    return(temp)

def linear_albedo_solver(mu1, mu2, guess_1, guess_2):
    """For the case of unstable solutions and ranges of the plot, we must have a stable linear albedo feeback solution.
    This solution is a modeling of a situation where solar intensity is decreased sufficiently until the earth 
    produces enough ice that the earth will reflect sufficient radiation to keep the earth in a stable ice age."""
    import numpy as np
    from scipy.optimize import fsolve
    deepfreeze = []
    for x in mu1:
        for y in mu2:
            solution = fsolve(linear_albedo_feedback, [guess_1, guess_2], args =(x,y))
            deepfreeze.append([solution[0], solution[1], x, y])
    deepfreeze = np.array(deepfreeze)
    return(deepfreeze)
    
